In [5]:
import pandas as pd
from functions.openai_functions import OpenAI_Client
from functions.db_functions import MySQL, PostGreSQL

In [6]:
openai = OpenAI_Client()
mysql = MySQL()
postgres = PostGreSQL()

In [3]:
df = mysql.load_chunk(chunk_size = 20)

In [4]:
embeddings = [openai.get_embedding(df.loc[i, 'string_for_embedding']) for i in range(len(df))]

In [5]:
df.head()

,id,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes,string_for_embedding,created_at
0,1,1,100,0,LAKES & RIVERS,,River mentioned most often in the Bible,the Jordan,1984-09-10,,"In round 1, with a value of 100, a daily doubl...",2025-05-01 18:17:12
1,2,1,200,0,LAKES & RIVERS,,Scottish word for lake,loch,1984-09-10,,"In round 1, with a value of 200, a daily doubl...",2025-05-01 18:17:12
2,3,1,400,0,LAKES & RIVERS,,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,,"In round 1, with a value of 400, a daily doubl...",2025-05-01 18:17:12
3,4,1,500,0,LAKES & RIVERS,,"Worlds largest lake, nearly 5 times as big as ...",the Caspian Sea,1984-09-10,,"In round 1, with a value of 500, a daily doubl...",2025-05-01 18:17:12
4,5,1,100,0,INVENTIONS,,Marconis wonderful wireless,a radio,1984-09-10,,"In round 1, with a value of 100, a daily doubl...",2025-05-01 18:17:12


In [6]:
df1 = pd.DataFrame({'id' : df['id'], 'content': df['string_for_embedding'], 'embedding': embeddings})

In [7]:
postgres.insert_data(df1, table_name = 'documents')

Inserted 20 rows into documents


In [9]:
prompt = "Are there any jeopardy questions related to Tiger Woods?"
embedding = openai.get_embedding(prompt)
prompt_vector = '[' + ','.join(map(str, embedding)) + ']'

In [10]:
import psycopg2
conn = psycopg2.connect(f"host={postgres.host} dbname={postgres.db_name} user={postgres.user} password={postgres.password}")
cursor = conn.cursor()

In [8]:
match_threshold = 0.7
match_count = 5

In [12]:
cursor = conn.cursor()
cursor.execute(
    'SELECT id, content '
    'FROM jeopardy_embeddings '
    'ORDER BY embedding <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': match_threshold, 'match_cnt': match_count}
    )
result = cursor.fetchall()
cursor.close()
for row in result:
    print(row)

(146953, 'In round 2, with a value of 600, a daily double value of 0, in the category FAMOUS NAMES, the question was In 1996 this 20-year-old African-American golfer won a record third consecutive U.S. Amateur title and the answer was Tiger Woods. This was on 1996-11-15. Notes: 1996 Celebrity Jeopardy! game 5..')
(178069, 'In round 1, with a value of 100, a daily double value of 0, in the category SPORTS STATS, the question was In 1997, his first full year on the PGA Tour, he became the first to exceed $2 million in earnings for a season and the answer was Tiger Woods. This was on 1999-03-22. Notes: 1999 Celebrity Jeopardy! Sports Anchors Night..')
(202602, 'In round 2, with a value of 400, a daily double value of 0, in the category HEY, SPORT!, the question was In Nov. 2000 Phil Mickelson caught this Tiger by the tail, winning the Tour Championship PGA tournament by 2 strokes and the answer was Tiger Woods. This was on 2001-02-06. Notes: 2001 Celebrity Jeopardy! game 2..')
(172622, 'I

In [ ]:

for row in result:
    print(row)

ProgrammingError: no results to fetch

In [63]:
cursor.close()

In [58]:
for partition in result.partitions():
    # partition is an iterable that will be at most 100 items
    for row in partition:
        print(f"{row}")

(1, '[1,2,3]')
(2, '[4,5,6]')


In [ ]:
query = ""

'In round 1, with a value of 200, a daily double value of 0, in the category LAKES & RIVERS, the question was Scottish word for lake and the answer was loch. This was on 1984-09-10.'

In [5]:
postgres.get_last_id(table_name = 'jeopardy_embeddings')

0

In [3]:
df = mysql.load_from_last_id(last_id = 354, chunk_size = 20)
df.head()

,id,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes,string_for_embedding,created_at
0,355,1,200,0,ISLANDS,,An Indonesian island or slang for coffee,Java,1984-09-19,,"In round 1, with a value of 200, a daily doubl...",2025-05-01 18:17:12
1,356,1,300,0,ISLANDS,,Original nest of Sam Spades falcon,Malta,1984-09-19,,"In round 1, with a value of 300, a daily doubl...",2025-05-01 18:17:12
2,357,1,400,0,ISLANDS,,The song 26 Miles Across The Sea overshoots th...,Catalina Island,1984-09-19,,"In round 1, with a value of 400, a daily doubl...",2025-05-01 18:17:12
3,358,1,100,0,TELEVISION,,"Nickname for the Pacific Princess, it sails ev...",The Love Boat,1984-09-19,,"In round 1, with a value of 100, a daily doubl...",2025-05-01 18:17:12
4,359,1,200,0,TELEVISION,,"Rhoda, Phyllis & Lou Grant were all spun off t...",The Mary Tyler Moore Show,1984-09-19,,"In round 1, with a value of 200, a daily doubl...",2025-05-01 18:17:12


In [4]:
openai.count_tokens("What jeopardy questions have been focused around technology, specifically that might use electromagnetic waves?")

16